# Working with virtualized resources on Grid'5000

When working with bare-metal machines isn't enough.

EnOSlib uses `Providers` to ... provide resources. They transform an abstract resource configuration into a concrete one.
To do so, they interact with an infrastructure where they get the resources from. There are different providers in EnOSlib: 

- Vbox/KVM to work with locally hosted virtual machines
- Openstack/Chameleon to work with bare-metal resources hosted in the Chameleon platform
- FiT/IOT lab to work with sensors or low profile machines
- **VmonG5k to work with virtual machines on Grid'5000**
- **Distem to work with lxc containers on Grid'5000**
- G5k, of course

The purpose of the above is to ease the use of the platform by internalizing some of the configuration tasks (e.g automatically managing the reservation on G5k, network configuration ...)

In the following we'll cover some of the EnOSlib way of managing virtual machines on Grid'5000, docker containers or lxc containers on Grid'5000.

---

- Website: https://discovery.gitlabpages.inria.fr/enoslib/index.html
- Instant chat: https://framateam.org/enoslib
- Source code: https://gitlab.inria.fr/discovery/enoslib

---

**Prerequisites:**

- A Grid'5000 account
- A working EnOSlib environment and Jupyter (not included in EnOSlib dependencies, but `pip install jupyterlab` will install it)




## Virtual Machines

The VMonG5K provider which provides a quick way to start virtual machines for you on Grid'5000. 

This setup is opinionated and follows these steps

- First, the number of required physical machine is computed and then they are reserved
- Second, a subnet is reserved (/22 or /16), which will be used as a pool of available MAC/IP 
- Third, the virtual machines are distributed on the physical machines and assigned a MAC/IP.
- Fourth, the virtual machines are started

---
The following configuration wil start 10 VMs with different roles.

In [9]:
import enoslib as en

# get some logging info
import logging
logging.basicConfig(level=logging.INFO)

# claim the resources
conf = (
    en.VMonG5kConf
    .from_settings(job_name="enoslib_providers")
    .add_machine(
        roles=["compute"],
        cluster="paravance",
        number=8,
        flavour_desc={
            "core": 2,
            "mem": 2048
        }
    )
    .add_machine(
        roles=["controler"],
        cluster="paravance",
        number=2,
        flavour="tiny"
    )
    .finalize()
)


provider = en.VMonG5k(conf)

roles, networks = provider.init()
print(roles)
print(networks)

INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from grenoble
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from lille
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from luxembourg
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from lyon
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from nancy
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from nantes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from sophia
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Submitting {'name': 'enoslib_providers', 'types': ['allow_classic_ssh'], 'resources': "{cluster='paravance'}/nodes=1+{cluster='paravance'}/nodes=1+slash_22=1,walltime=02:00:00", 'command': 'sleep 31536000', 'queue': 'default'} on rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Waiting for 1817598 o


PLAY [This is a play for virtual machines on Grid'5000] ***************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [paravance-40.rennes.grid5000.fr]
ok: [paravance-3.rennes.grid5000.fr]

TASK [Enable nested virtualization] ***********************************************************************************************************************
changed: [paravance-3.rennes.grid5000.fr]
changed: [paravance-40.rennes.grid5000.fr]

TASK [list only running VMs] ******************************************************************************************************************************
ok: [paravance-3.rennes.grid5000.fr]
ok: [paravance-40.rennes.grid5000.fr]

TASK [debug] ***************************************************************************************************************************

In [10]:
roles

{'compute': [VirtualMachine(address='10.158.16.2', alias='virtual-158-16-2', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices=''), VirtualMachine(address='10.158.16.3', alias='virtual-158-16-3', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices=''), VirtualMachine(address='10.158.16.4', alias='virtual-158-16-4', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices=''), VirtualMachine(address='10.158.16.5', alias='virtual-158-16-5', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices=''), VirtualMachine(address='10.158.16.6', alias='virtual-158-16-6', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices=''), VirtualMachine(address='10.158.16.7', alias='virtual-158-16-7', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices=''), VirtualMachine(address='10.158.16.8', alias='virtual-158-16-8', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices=''), VirtualMachine(address='10.158.16.9', alias='virtual-158-16-9', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices='')], 'controler': [VirtualMachine(address='10.158.16.10', alias='virtual-158-16-10', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices=''), VirtualMachine(address='10.158.16.11', alias='virtual-158-16-11', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices='')]}

In [11]:
networks

{'__subnet__': [<enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f87b41e6490>]}

In [12]:
# VMs can take some time to be reachable, let's wait for them
en.wait_for(roles)
roles = en.sync_info(roles, networks)


PLAY [all] ************************************************************************************************************************************************

TASK [hostname] *******************************************************************************************************************************************
 [started TASK: hostname on virtual-158-16-2]
 [started TASK: hostname on virtual-158-16-3]
 [started TASK: hostname on virtual-158-16-4]
 [started TASK: hostname on virtual-158-16-5]
 [started TASK: hostname on virtual-158-16-6]
 [started TASK: hostname on virtual-158-16-7]
 [started TASK: hostname on virtual-158-16-8]
 [started TASK: hostname on virtual-158-16-9]
 [started TASK: hostname on virtual-158-16-10]
 [started TASK: hostname on virtual-158-16-11]


fatal: [virtual-158-16-2]: UNREACHABLE! => {"changed": false, "msg": "Failed to connect to the host via ssh: Warning: Permanently added 'access.grid5000.fr,194.254.60.33' (ECDSA) to the list of known hosts.\r\nchannel 0: open failed: connect failed: Connection refused\r\nstdio forwarding failed\r\nssh_exchange_identification: Connection closed by remote host", "unreachable": true}
fatal: [virtual-158-16-11]: UNREACHABLE! => {"changed": false, "msg": "Failed to connect to the host via ssh: Warning: Permanently added 'access.grid5000.fr,194.254.60.33' (ECDSA) to the list of known hosts.\r\nchannel 0: open failed: connect failed: Connection refused\r\nstdio forwarding failed\r\nssh_exchange_identification: Connection closed by remote host", "unreachable": true}
fatal: [virtual-158-16-4]: UNREACHABLE! => {"changed": false, "msg": "Failed to connect to the host via ssh: Warning: Permanently added 'access.grid5000.fr,194.254.60.33' (ECDSA) to the list of known hosts.\r\nchannel 0: open faile

changed: [virtual-158-16-10]


fatal: [virtual-158-16-5]: UNREACHABLE! => {"changed": false, "msg": "Failed to connect to the host via ssh: Warning: Permanently added 'access.grid5000.fr,194.254.60.33' (ECDSA) to the list of known hosts.\r\nchannel 0: open failed: connect failed: Connection refused\r\nstdio forwarding failed\r\nssh_exchange_identification: Connection closed by remote host", "unreachable": true}
fatal: [virtual-158-16-6]: UNREACHABLE! => {"changed": false, "msg": "Failed to connect to the host via ssh: Warning: Permanently added 'access.grid5000.fr,194.254.60.33' (ECDSA) to the list of known hosts.\r\nchannel 0: open failed: connect failed: Connection refused\r\nstdio forwarding failed\r\nssh_exchange_identification: Connection closed by remote host", "unreachable": true}
fatal: [virtual-158-16-7]: UNREACHABLE! => {"changed": false, "msg": "Failed to connect to the host via ssh: Warning: Permanently added 'access.grid5000.fr,194.254.60.33' (ECDSA) to the list of known hosts.\r\nchannel 0: open failed


PLAY [all] ************************************************************************************************************************************************

TASK [hostname] *******************************************************************************************************************************************
 [started TASK: hostname on virtual-158-16-2]
 [started TASK: hostname on virtual-158-16-3]
 [started TASK: hostname on virtual-158-16-4]
 [started TASK: hostname on virtual-158-16-5]
 [started TASK: hostname on virtual-158-16-6]
 [started TASK: hostname on virtual-158-16-7]
 [started TASK: hostname on virtual-158-16-8]
 [started TASK: hostname on virtual-158-16-9]
 [started TASK: hostname on virtual-158-16-10]
 [started TASK: hostname on virtual-158-16-11]
changed: [virtual-158-16-10]
changed: [virtual-158-16-2]
changed: [virtual-158-16-3]
changed: [virtual-158-16-5]
changed: [virtual-158-16-4]
changed: [virtual-158-16-6]
changed: [virtual-158-16-7]
changed: [virtual-158-16-9]

In [13]:
roles

ip
fe80::216:3eff:fe9e:1002/64
10.158.16.2/14
ip
::1/128
127.0.0.1/8
ip
fe80::216:3eff:fe9e:1003/64
10.158.16.3/14
ip
::1/128


In [14]:
networks

{'__subnet__': [<enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f87b41e6490>]}

In [15]:
results = en.run_command("nproc", roles=roles)
[(r.host,  r.stdout) for r in results]


PLAY [all] ************************************************************************************************************************************************

TASK [nproc] **********************************************************************************************************************************************
 [started TASK: nproc on virtual-158-16-2]
 [started TASK: nproc on virtual-158-16-3]
 [started TASK: nproc on virtual-158-16-4]
 [started TASK: nproc on virtual-158-16-5]
 [started TASK: nproc on virtual-158-16-6]
 [started TASK: nproc on virtual-158-16-7]
 [started TASK: nproc on virtual-158-16-8]
 [started TASK: nproc on virtual-158-16-9]
 [started TASK: nproc on virtual-158-16-10]
 [started TASK: nproc on virtual-158-16-11]
changed: [virtual-158-16-2]
changed: [virtual-158-16-3]
changed: [virtual-158-16-4]
changed: [virtual-158-16-5]
changed: [virtual-158-16-6]
changed: [virtual-158-16-7]
changed: [virtual-158-16-8]
changed: [virtual-158-16-9]
changed: [virtual-158-16-10]
c

[('virtual-158-16-2', '2'),
 ('virtual-158-16-3', '2'),
 ('virtual-158-16-4', '2'),
 ('virtual-158-16-5', '2'),
 ('virtual-158-16-6', '2'),
 ('virtual-158-16-7', '2'),
 ('virtual-158-16-8', '2'),
 ('virtual-158-16-9', '2'),
 ('virtual-158-16-10', '1'),
 ('virtual-158-16-11', '1')]

In [16]:
provider.destroy()

INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from grenoble
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from lille
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from luxembourg
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from lyon
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from nancy
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from nantes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading 1817598 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from sophia
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Killing the job (rennes, 1817598)


## Docker containers

There's no specific provider for manipulating Docker Container as hosts but some utility functions to help you.

- most of the time you don't need to change the internal state of docker containers 
  (you just fire up some applications using docker container and that's it)
- so this technique cover use cases where you need to use docker container instead a bare metal machine
  and needs to interact with the containerized linux distribution (installing software, configuring stuffs ...)
  
---
The strategy is the following:
- First reserve some bare-metal machines
- Install and start some docker containers
- Get the representation of the docker containers as Hosts

In [14]:
import logging
from pathlib import Path

import enoslib as en

logging.basicConfig(level=logging.DEBUG)

prod_network = en.G5kNetworkConf(
    id="n1", type="prod", roles=["my_network"], site="rennes"
)
conf = (
    en.G5kConf.from_settings(job_name="enoslib_docker", job_type="allow_classic_ssh")
    .add_network_conf(prod_network)
    .add_machine(
        roles=["control"], cluster="paravance", nodes=1, primary_network=prod_network
    )
    .finalize()
)

provider = en.G5k(conf)

# Get actual resources
roles, networks = provider.init()

DEBUG:enoslib.infra.enos_g5k.g5k_api_utils:Reloading get_all_clusters_sites from cachedir
DEBUG:enoslib.infra.configuration:{
    "dhcp": true,
    "force_deploy": false,
    "env_name": "debian10-x64-nfs",
    "job_name": "enoslib_docker",
    "job_type": "allow_classic_ssh",
    "key": "/home/msimonin/.ssh/id_rsa.pub",
    "queue": "default",
    "walltime": "02:00:00",
    "resources": {
        "machines": [
            {
                "roles": [
                    "control"
                ],
                "primary_network": "n1",
                "secondary_networks": [],
                "cluster": "paravance",
                "nodes": 1
            }
        ],
        "networks": [
            {
                "id": "n1",
                "type": "prod",
                "roles": [
                    "my_network"
                ],
                "site": "rennes"
            }
        ]
    }
}
DEBUG:enoslib.infra.enos_g5k.driver:Loading the OargridDynamicDriver
DEBUG:urll

We install docker using the EnOSlib's service.

In [15]:
# Workaround
with en.actions(roles=roles) as a:
    a.file(path="/etc/apt/sources.list.d/repo.radeon.com.list", state="absent")
 
# Install docker
d = en.Docker(agent=roles["control"], bind_var_docker="/tmp/docker")
d.deploy()

DEBUG:enoslib.api:{'hosts': 'all', 'tasks': [{'name': 'file', 'file': {'path': '/etc/apt/sources.list.d/repo.radeon.com.list', 'state': 'absent'}}], 'gather_facts': False, 'strategy': 'linear'}
DEBUG:enoslib.api:{'hosts': 'all', 'tasks': [{'name': 'file', 'file': {'path': '/etc/apt/sources.list.d/repo.radeon.com.list', 'state': 'absent'}}], 'gather_facts': False, 'strategy': 'linear'}
DEBUG:enoslib.api:Using extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ************************************************************************************************************************************************

TASK [file] ***********************************************************************************************************************************************
 [started TASK: file on paravance-3.rennes.grid5000.fr]
changed: [paravance-3.rennes.grid5000.fr]


DEBUG:enoslib.api:Using extra_vars = {'registry': {'type': 'none'}, 'enos_action': 'deploy', 'swarm': False, 'bind_var_docker': '/tmp/docker', 'ansible_python_interpreter': 'python3'}
DEBUG:enoslib.api:Running playbook /home/msimonin/workspace/repos/enoslib/enoslib/service/docker/docker.yml with vars:
{'registry': {'type': 'none'}, 'enos_action': 'deploy', 'swarm': False, 'bind_var_docker': '/tmp/docker', 'ansible_python_interpreter': 'python3'}



PLAY [Deploy docker and registry] *************************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [paravance-3.rennes.grid5000.fr]

TASK [registry : include] *********************************************************************************************************************************
included: /home/msimonin/workspace/repos/enoslib/enoslib/service/docker/roles/registry/tasks/deploy.yml for paravance-3.rennes.grid5000.fr

TASK [registry : include] *********************************************************************************************************************************
included: /home/msimonin/workspace/repos/enoslib/enoslib/service/docker/roles/registry/tasks/none/agent.yml for paravance-3.rennes.grid5000.fr

TASK [registry : include] **************************

[WARNING]: Updating cache and auto-installing missing dependency: python3-apt


ok: [paravance-3.rennes.grid5000.fr]

TASK [registry : fix "No module named ssl_match_hostname" issue] ******************************************************************************************
changed: [paravance-3.rennes.grid5000.fr]

TASK [registry : Installing docker python bindings] *******************************************************************************************************
changed: [paravance-3.rennes.grid5000.fr]

TASK [registry : Creating docker state directory] *********************************************************************************************************
changed: [paravance-3.rennes.grid5000.fr] => (item=/tmp/docker)
changed: [paravance-3.rennes.grid5000.fr] => (item=/var/lib/docker)

TASK [registry : Bind mount the docker volume directory] **************************************************************************************************
changed: [paravance-3.rennes.grid5000.fr]

TASK [registry : Installing docker] ****************************************

In [16]:
# Start some containers
N = 5
with en.play_on(roles=roles) as p:
    for i in range(N):
        p.docker_container(
            name=f"mydocker-{i}",
            image="ubuntu",
            state="started",
            command="sleep 10d",
        )

DEBUG:enoslib.api:{'hosts': 'all', 'tasks': [{'name': 'docker_container', 'docker_container': {'name': 'mydocker-0', 'image': 'ubuntu', 'state': 'started', 'command': 'sleep 10d'}}, {'name': 'docker_container', 'docker_container': {'name': 'mydocker-1', 'image': 'ubuntu', 'state': 'started', 'command': 'sleep 10d'}}, {'name': 'docker_container', 'docker_container': {'name': 'mydocker-2', 'image': 'ubuntu', 'state': 'started', 'command': 'sleep 10d'}}, {'name': 'docker_container', 'docker_container': {'name': 'mydocker-3', 'image': 'ubuntu', 'state': 'started', 'command': 'sleep 10d'}}, {'name': 'docker_container', 'docker_container': {'name': 'mydocker-4', 'image': 'ubuntu', 'state': 'started', 'command': 'sleep 10d'}}], 'gather_facts': False, 'strategy': 'linear'}
DEBUG:enoslib.api:{'hosts': 'all', 'tasks': [{'name': 'docker_container', 'docker_container': {'name': 'mydocker-0', 'image': 'ubuntu', 'state': 'started', 'command': 'sleep 10d'}}, {'name': 'docker_container', 'docker_conta


PLAY [all] ************************************************************************************************************************************************

TASK [docker_container] ***********************************************************************************************************************************
 [started TASK: docker_container on paravance-3.rennes.grid5000.fr]
changed: [paravance-3.rennes.grid5000.fr]

TASK [docker_container] ***********************************************************************************************************************************
 [started TASK: docker_container on paravance-3.rennes.grid5000.fr]
changed: [paravance-3.rennes.grid5000.fr]

TASK [docker_container] ***********************************************************************************************************************************
 [started TASK: docker_container on paravance-3.rennes.grid5000.fr]
changed: [paravance-3.rennes.grid5000.fr]

TASK [docker_container] *****************

In [17]:
dockers = en.get_dockers(roles=roles)

DEBUG:enoslib.api:{'hosts': '*', 'gather_facts': False, 'tasks': [{'name': 'docker ps -q --filter name=.* | xargs docker inspect', 'shell': 'docker ps -q --filter name=.* | xargs docker inspect', 'args': {}}]}
DEBUG:enoslib.api:Using extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [*] **************************************************************************************************************************************************

TASK [docker ps -q --filter name=.* | xargs docker inspect] ***********************************************************************************************
 [started TASK: docker ps -q --filter name=.* | xargs docker inspect on paravance-3.rennes.grid5000.fr]
changed: [paravance-3.rennes.grid5000.fr]


In [18]:
dockers

[DockerHost(address='/mydocker-4', alias='/mydocker-4-paravance-3.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={'ansible_connection': 'docker', 'ansible_docker_extra_args': '-H ssh://root@paravance-3.rennes.grid5000.fr', 'mitogen_via': 'root@paravance-3.rennes.grid5000.fr'}, net_devices=set()),
 DockerHost(address='/mydocker-3', alias='/mydocker-3-paravance-3.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={'ansible_connection': 'docker', 'ansible_docker_extra_args': '-H ssh://root@paravance-3.rennes.grid5000.fr', 'mitogen_via': 'root@paravance-3.rennes.grid5000.fr'}, net_devices=set()),
 DockerHost(address='/mydocker-2', alias='/mydocker-2-paravance-3.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={'ansible_connection': 'docker', 'ansible_docker_extra_args': '-H ssh://root@paravance-3.rennes.grid5000.fr', 'mitogen_via': 'root@paravance-3.rennes.grid5000.fr'}, net_devices=set()),
 DockerHost(address='/mydocker-1', alias='/mydocker-

EnOSlib/Ansible requires python at the destination for many operations. However minimal docker images will lickely not include python...
Using the option `raw=True` can overcome this limitation: https://docs.ansible.com/ansible/latest/collections/ansible/builtin/raw_module.html

In [19]:
en.run_command("hostname", roles=dockers, raw=True)

DEBUG:enoslib.api:{'hosts': 'all', 'gather_facts': False, 'tasks': [{'name': 'hostname', 'raw': 'hostname', 'args': {}}]}
DEBUG:enoslib.api:Using extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ************************************************************************************************************************************************

TASK [hostname] *******************************************************************************************************************************************
 [started TASK: hostname on /mydocker-4-paravance-3.rennes.grid5000.fr]
 [started TASK: hostname on /mydocker-3-paravance-3.rennes.grid5000.fr]
 [started TASK: hostname on /mydocker-2-paravance-3.rennes.grid5000.fr]
 [started TASK: hostname on /mydocker-1-paravance-3.rennes.grid5000.fr]
 [started TASK: hostname on /mydocker-0-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-3-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-1-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-0-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-2-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-4-paravance-3.rennes.grid5000.fr]


CommandResult(host='/mydocker-3-paravance-3.rennes.grid5000.fr', task='hostname', status='OK', payload={'rc': 0, 'stdout': '14ec738136d2\n', 'stdout_lines': ['14ec738136d2'], 'stderr': '', 'stderr_lines': [], 'changed': True, '_ansible_no_log': False})
CommandResult(host='/mydocker-1-paravance-3.rennes.grid5000.fr', task='hostname', status='OK', payload={'rc': 0, 'stdout': 'ded79a811a0b\n', 'stdout_lines': ['ded79a811a0b'], 'stderr': '', 'stderr_lines': [], 'changed': True, '_ansible_no_log': False})
CommandResult(host='/mydocker-0-paravance-3.rennes.grid5000.fr', task='hostname', status='OK', payload={'rc': 0, 'stdout': '57d152421527\n', 'stdout_lines': ['57d152421527'], 'stderr': '', 'stderr_lines': [], 'changed': True, '_ansible_no_log': False})
CommandResult(host='/mydocker-2-paravance-3.rennes.grid5000.fr', task='hostname', status='OK', payload={'rc': 0, 'stdout': 'b731fd8a5574\n', 'stdout_lines': ['b731fd8a5574'], 'stderr': '', 'stderr_lines': [], 'changed': True, '_ansible_no_log': False})
CommandResult(host='/mydocker-4-paravance-3.rennes.grid5000.fr', task='hostname', status='OK', payload={'rc': 0, 'stdout': 'feeb26ebb5ca\n', 'stdout_lines': ['feeb26ebb5ca'], 'stderr': '', 'stderr_lines': [], 'changed': True, '_ansible_no_log': False})

In [20]:
en.run_command("apt update && apt install -y python3", roles=dockers, raw=True)

DEBUG:enoslib.api:{'hosts': 'all', 'gather_facts': False, 'tasks': [{'name': 'apt update && apt install -y python3', 'raw': 'apt update && apt install -y python3', 'args': {}}]}
DEBUG:enoslib.api:Using extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ************************************************************************************************************************************************

TASK [apt update && apt install -y python3] ***************************************************************************************************************
 [started TASK: apt update && apt install -y python3 on /mydocker-4-paravance-3.rennes.grid5000.fr]
 [started TASK: apt update && apt install -y python3 on /mydocker-3-paravance-3.rennes.grid5000.fr]
 [started TASK: apt update && apt install -y python3 on /mydocker-2-paravance-3.rennes.grid5000.fr]
 [started TASK: apt update && apt install -y python3 on /mydocker-1-paravance-3.rennes.grid5000.fr]
 [started TASK: apt update && apt install -y python3 on /mydocker-0-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-2-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-1-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-4-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-0-pa

CommandResult(host='/mydocker-2-paravance-3.rennes.grid5000.fr', task='apt update && apt install -y python3', status='OK', payload={'rc': 0, 'stdout': "Get:1 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]\nGet:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]\nGet:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]\nGet:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [101 kB]\nGet:5 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]\nGet:6 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [177 kB]\nGet:7 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]\nGet:8 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages [1275 kB]\nGet:9 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [30.1 kB]\nGet:10 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [575 kB]\nGet:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [1514 kB]\nGet:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1069 kB]\nGet:13 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [1070 kB]\nGet:14 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [33.3 kB]\nGet:15 http://archive.ubuntu.com/ubuntu focal-backports/universe amd64 Packages [6324 B]\nGet:16 http://archive.ubuntu.com/ubuntu focal-backports/main amd64 Packages [2668 B]\nGet:17 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [790 kB]\nGet:18 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [525 kB]\nFetched 19.0 MB in 2s (11.2 MB/s)\nReading package lists...\nBuilding dependency tree...\nReading state information...\nAll packages are up to date.\nReading package lists...\nBuilding dependency tree...\nReading state information...\nThe following additional packages will be installed:\n  file libexpat1 libmagic-mgc libmagic1 libmpdec2 libpython3-stdlib\n  libpython3.8-minimal libpython3.8-stdlib libreadline8 libsqlite3-0 libssl1.1\n  mime-support python3-minimal python3.8 python3.8-minimal readline-common\n  xz-utils\nSuggested packages:\n  python3-doc python3-tk python3-venv python3.8-venv python3.8-doc binutils\n  binfmt-support readline-doc\nThe following NEW packages will be installed:\n  file libexpat1 libmagic-mgc libmagic1 libmpdec2 libpython3-stdlib\n  libpython3.8-minimal libpython3.8-stdlib libreadline8 libsqlite3-0 libssl1.1\n  mime-support python3 python3-minimal python3.8 python3.8-minimal\n  readline-common xz-utils\n0 upgraded, 18 newly installed, 0 to remove and 0 not upgraded.\nNeed to get 7393 kB of archives.\nAfter this operation, 32.8 MB of additional disk space will be used.\nGet:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libssl1.1 amd64 1.1.1f-1ubuntu2.8 [1320 kB]\nGet:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpython3.8-minimal amd64 3.8.10-0ubuntu1~20.04 [717 kB]\nGet:3 http://archive.ubuntu.com/ubuntu focal/main amd64 libexpat1 amd64 2.2.9-1build1 [73.3 kB]\nGet:4 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 python3.8-minimal amd64 3.8.10-0ubuntu1~20.04 [1898 kB]\nGet:5 http://archive.ubuntu.com/ubuntu focal/main amd64 python3-minimal amd64 3.8.2-0ubuntu2 [23.6 kB]\nGet:6 http://archive.ubuntu.com/ubuntu focal/main amd64 mime-support all 3.64ubuntu1 [30.6 kB]\nGet:7 http://archive.ubuntu.com/ubuntu focal/main amd64 libmpdec2 amd64 2.4.2-3 [81.1 kB]\nGet:8 http://archive.ubuntu.com/ubuntu focal/main amd64 readline-common all 8.0-4 [53.5 kB]\nGet:9 http://archive.ubuntu.com/ubuntu focal/main amd64 libreadline8 amd64 8.0-4 [131 kB]\nGet:10 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libsqlite3-0 amd64 3.31.1-4ubuntu0.2 [549 kB]\nGet:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpython3.8-stdlib amd64 3.8.10-0ubuntu1~20.04 [1675 kB]\nGet:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 python3

In [21]:
# we're now good to use raw=False (the default)
en.run_command("date", roles=dockers)

DEBUG:enoslib.api:{'hosts': 'all', 'gather_facts': False, 'tasks': [{'name': 'date', 'shell': 'date', 'args': {}}]}
DEBUG:enoslib.api:Using extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ************************************************************************************************************************************************

TASK [date] ***********************************************************************************************************************************************
 [started TASK: date on /mydocker-4-paravance-3.rennes.grid5000.fr]
 [started TASK: date on /mydocker-3-paravance-3.rennes.grid5000.fr]
 [started TASK: date on /mydocker-2-paravance-3.rennes.grid5000.fr]
 [started TASK: date on /mydocker-1-paravance-3.rennes.grid5000.fr]
 [started TASK: date on /mydocker-0-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-4-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-3-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-2-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-0-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-1-paravance-3.rennes.grid5000.fr]


CommandResult(host='/mydocker-4-paravance-3.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Wed Sep  8 08:13:35 UTC 2021', 'stderr': '', 'rc': 0, 'start': '2021-09-08 08:13:35.607749', 'end': '2021-09-08 08:13:35.610890', 'delta': '0:00:00.003141', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Wed Sep  8 08:13:35 UTC 2021'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='/mydocker-3-paravance-3.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Wed Sep  8 08:13:35 UTC 2021', 'stderr': '', 'rc': 0, 'start': '2021-09-08 08:13:35.623828', 'end': '2021-09-08 08:13:35.626856', 'delta': '0:00:00.003028', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Wed Sep  8 08:13:35 UTC 2021'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='/mydocker-2-paravance-3.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Wed Sep  8 08:13:35 UTC 2021', 'stderr': '', 'rc': 0, 'start': '2021-09-08 08:13:35.657018', 'end': '2021-09-08 08:13:35.659804', 'delta': '0:00:00.002786', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Wed Sep  8 08:13:35 UTC 2021'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='/mydocker-0-paravance-3.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Wed Sep  8 08:13:35 UTC 2021', 'stderr': '', 'rc': 0, 'start': '2021-09-08 08:13:35.800404', 'end': '2021-09-08 08:13:35.803247', 'delta': '0:00:00.002843', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Wed Sep  8 08:13:35 UTC 2021'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='/mydocker-1-paravance-3.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Wed Sep  8 08:13:35 UTC 2021', 'stderr': '', 'rc': 0, 'start': '2021-09-08 08:13:35.816079', 'end': '2021-09-08 08:13:35.818793', 'delta': '0:00:00.002714', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Wed Sep  8 08:13:35 UTC 2021'], 'stderr_lines': [], '_ansible_no_log': False})

In [23]:
en.run_command("apt install -y iproute2", roles=dockers)


DEBUG:enoslib.api:{'hosts': 'all', 'gather_facts': False, 'tasks': [{'name': 'apt install -y iproute2', 'shell': 'apt install -y iproute2', 'args': {}}]}
DEBUG:enoslib.api:Using extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ************************************************************************************************************************************************

TASK [apt install -y iproute2] ****************************************************************************************************************************
 [started TASK: apt install -y iproute2 on /mydocker-4-paravance-3.rennes.grid5000.fr]
 [started TASK: apt install -y iproute2 on /mydocker-3-paravance-3.rennes.grid5000.fr]
 [started TASK: apt install -y iproute2 on /mydocker-2-paravance-3.rennes.grid5000.fr]
 [started TASK: apt install -y iproute2 on /mydocker-1-paravance-3.rennes.grid5000.fr]
 [started TASK: apt install -y iproute2 on /mydocker-0-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-1-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-4-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-3-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-2-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-0-paravance-3.r

CommandResult(host='/mydocker-1-paravance-3.rennes.grid5000.fr', task='apt install -y iproute2', status='OK', payload={'cmd': 'apt install -y iproute2', 'stdout': 'Reading package lists...\nBuilding dependency tree...\nReading state information...\niproute2 is already the newest version (5.5.0-1ubuntu1).\n0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.', 'stderr': '\nWARNING: apt does not have a stable CLI interface. Use with caution in scripts.', 'rc': 0, 'start': '2021-09-08 08:17:01.391583', 'end': '2021-09-08 08:17:02.426735', 'delta': '0:00:01.035152', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'apt install -y iproute2', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Reading package lists...', 'Building dependency tree...', 'Reading state information...', 'iproute2 is already the newest version (5.5.0-1ubuntu1).', '0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.'], 'stderr_lines': ['', 'WARNING: apt does not have a stable CLI interface. Use with caution in scripts.'], '_ansible_no_log': False})
CommandResult(host='/mydocker-4-paravance-3.rennes.grid5000.fr', task='apt install -y iproute2', status='OK', payload={'cmd': 'apt install -y iproute2', 'stdout': 'Reading package lists...\nBuilding dependency tree...\nReading state information...\niproute2 is already the newest version (5.5.0-1ubuntu1).\n0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.', 'stderr': '\nWARNING: apt does not have a stable CLI interface. Use with caution in scripts.', 'rc': 0, 'start': '2021-09-08 08:17:01.356212', 'end': '2021-09-08 08:17:02.461619', 'delta': '0:00:01.105407', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'apt install -y iproute2', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Reading package lists...', 'Building dependency tree...', 'Reading state information...', 'iproute2 is already the newest version (5.5.0-1ubuntu1).', '0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.'], 'stderr_lines': ['', 'WARNING: apt does not have a stable CLI interface. Use with caution in scripts.'], '_ansible_no_log': False})
CommandResult(host='/mydocker-3-paravance-3.rennes.grid5000.fr', task='apt install -y iproute2', status='OK', payload={'cmd': 'apt install -y iproute2', 'stdout': 'Reading package lists...\nBuilding dependency tree...\nReading state information...\niproute2 is already the newest version (5.5.0-1ubuntu1).\n0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.', 'stderr': '\nWARNING: apt does not have a stable CLI interface. Use with caution in scripts.', 'rc': 0, 'start': '2021-09-08 08:17:01.419484', 'end': '2021-09-08 08:17:02.466031', 'delta': '0:00:01.046547', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'apt install -y iproute2', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Reading package lists...', 'Building dependency tree...', 'Reading state information...', 'iproute2 is already the newest version (5.5.0-1ubuntu1).', '0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.'], 'stderr_lines': ['', 'WARNING: apt does not have a stable CLI interface. Use with caution in scripts.'], '_ansible_no_log': False})
CommandResult(host='/mydocker-2-paravance-3.rennes.grid5000.fr', task='apt install -y iproute2', status='OK', payload={'cmd': 'apt install -y iproute2', 'stdout': 'Reading package lists...\nBuilding dependency tree...\nReading state information...\niproute2 is already the newest version (5.5.0-1ubuntu1).\n0 upgraded, 0 newly i

In [24]:
results = en.run_command("ip a", roles=dockers)
for r in results:
    print(r.host)
    print("-"*20)
    print(r.stdout)

DEBUG:enoslib.api:{'hosts': 'all', 'gather_facts': False, 'tasks': [{'name': 'ip a', 'shell': 'ip a', 'args': {}}]}
DEBUG:enoslib.api:Using extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ************************************************************************************************************************************************

TASK [ip a] ***********************************************************************************************************************************************
 [started TASK: ip a on /mydocker-4-paravance-3.rennes.grid5000.fr]
 [started TASK: ip a on /mydocker-3-paravance-3.rennes.grid5000.fr]
 [started TASK: ip a on /mydocker-2-paravance-3.rennes.grid5000.fr]
 [started TASK: ip a on /mydocker-1-paravance-3.rennes.grid5000.fr]
 [started TASK: ip a on /mydocker-0-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-2-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-4-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-3-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-0-paravance-3.rennes.grid5000.fr]
changed: [/mydocker-1-paravance-3.rennes.grid5000.fr]
/mydocker-2-paravance-3.rennes.grid5000.fr
--------------------
1: lo: <LOOP

In [25]:
provider.destroy()

DEBUG:urllib3.connectionpool:Resetting dropped connection: api.grid5000.fr
DEBUG:urllib3.connectionpool:https://api.grid5000.fr:443 "GET /stable/sites HTTP/1.1" 200 30688
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_docker from grenoble
DEBUG:urllib3.connectionpool:https://api.grid5000.fr:443 "GET /stable/sites/grenoble/jobs?name=enoslib_docker&state=waiting%2Claunching%2Crunning&user=msimonin HTTP/1.1" 200 233
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_docker from lille
DEBUG:urllib3.connectionpool:https://api.grid5000.fr:443 "GET /stable/sites/lille/jobs?name=enoslib_docker&state=waiting%2Claunching%2Crunning&user=msimonin HTTP/1.1" 200 227
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_docker from luxembourg
DEBUG:urllib3.connectionpool:https://api.grid5000.fr:443 "GET /stable/sites/luxembourg/jobs?name=enoslib_docker&state=waiting%2Claunching%2Crunning&user=msimonin HTTP/1.1" 200 237
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading en

## LXC containers with Distem

https://distem.gitlabpages.inria.fr/

EnOSlib offers a way to use Distem on Grid'5000 using a dedicated provider.
This provider will do the heavy-lifting of 

- reserving the physical resources needed (some machines and a subnet)
- deploying a linux environment (Distem requires it)
- starting the distem server and agents
- makes the initial API calls to the distem server to start the wanted containers.



> Since the Distem server is accessible only from inside Grid'5000, to use this provider it's recommended to launch the script **from inside   
  Grid'5000**. 
  But if you're adventurous and outside Grid'5000, you can try to first create a proxy SOCKS to the frontend of your choice and then set the relevant   variable in your environment (and set `verify: False` in your `~/.python-grid5000.yaml`, and ... restart the kernel. Yes, notebooks are wild sometimes !)

In the following we'll consider being **outside** Grid'5000

In [1]:
# in a terminal: 
# ssh -ND 2100 rennes.g5k
import os
os.environ["http_proxy"] = "socks5h://localhost:2100"
os.environ["https_proxy"] = "socks5h://localhost:2100"

In [2]:
import logging
from pathlib import Path

import enoslib as en

FORCE = False
CLUSTER = "paravance"

logging.basicConfig(level=logging.DEBUG)
en.config.set_config(g5k_cache_dir="plop")
# claim the resources
conf = (
    en.DistemConf
    .from_settings(
        job_name="enoslib_distem",
        force_deploy=FORCE,
        image="file:///home/msimonin/public/distem-stretch.tgz"
    )
    .add_machine(
        roles=["server"],
        cluster=CLUSTER,
        number=1,
        flavour="large"
    )
    .add_machine(
        roles=["client"],
        cluster=CLUSTER,
        number=1,
        flavour="large"
    )
    .finalize()
)

provider = en.Distem(conf)
conf

DEBUG:enoslib.infra.configuration:{
    "job_name": "enoslib_distem",
    "queue": "default",
    "walltime": "02:00:00",
    "force_deploy": false,
    "image": "file:///home/msimonin/public/distem-stretch.tgz",
    "resources": {
        "machines": [
            {
                "undercloud": [],
                "cluster": "paravance",
                "roles": [
                    "server"
                ],
                "flavour_desc": {
                    "core": 4,
                    "mem": 4096
                },
                "number": 1
            },
            {
                "undercloud": [],
                "cluster": "paravance",
                "roles": [
                    "client"
                ],
                "flavour_desc": {
                    "core": 4,
                    "mem": 4096
                },
                "number": 1
            }
        ],
        "networks": [
            "enos_network"
        ]
    }
}


Conf@0x7f3fe9717950
{
    "job_name": "enoslib_distem",
    "queue": "default",
    "walltime": "02:00:00",
    "force_deploy": false,
    "image": "file:///home/msimonin/public/distem-stretch.tgz",
    "resources": {
        "machines": [
            {
                "undercloud": [],
                "cluster": "paravance",
                "roles": [
                    "server"
                ],
                "flavour_desc": {
                    "core": 4,
                    "mem": 4096
                },
                "number": 1
            },
            {
                "undercloud": [],
                "cluster": "paravance",
                "roles": [
                    "client"
                ],
                "flavour_desc": {
                    "core": 4,
                    "mem": 4096
                },
                "number": 1
            }
        ],
        "networks": [
            "enos_network"
        ]
    }
}

In [6]:
roles, networks = provider.init()

DEBUG:enoslib.infra.enos_g5k.g5k_api_utils:Reloading get_all_clusters_sites from plop
DEBUG:enoslib.infra.enos_g5k.g5k_api_utils:Reloading get_all_clusters_sites from plop
DEBUG:urllib3.connectionpool:Resetting dropped connection: api.grid5000.fr
DEBUG:urllib3.connectionpool:https://api.grid5000.fr:443 "GET /stable/sites/rennes HTTP/1.1" 200 3814
DEBUG:urllib3.connectionpool:https://api.grid5000.fr:443 "GET /stable/sites/rennes/clusters/paravance HTTP/1.1" 200 10339
DEBUG:urllib3.connectionpool:https://api.grid5000.fr:443 "GET /stable/sites/rennes/clusters/paravance/nodes HTTP/1.1" 200 485226
DEBUG:enoslib.infra.enos_g5k.g5k_api_utils:Reloading get_all_clusters_sites from plop
DEBUG:enoslib.infra.enos_g5k.g5k_api_utils:Reloading get_all_clusters_sites from plop
DEBUG:urllib3.connectionpool:https://api.grid5000.fr:443 "GET /stable/sites/rennes HTTP/1.1" 200 3814
DEBUG:urllib3.connectionpool:https://api.grid5000.fr:443 "GET /stable/sites/rennes/clusters/paravance HTTP/1.1" 200 10339
DEBU


PLAY [all] ************************************************************************************************************************************************

TASK [copy] ***********************************************************************************************************************************************
 [started TASK: copy on paravance-12.rennes.grid5000.fr]
 [started TASK: copy on paravance-13.rennes.grid5000.fr]
changed: [paravance-13.rennes.grid5000.fr]
changed: [paravance-12.rennes.grid5000.fr]

TASK [copy] ***********************************************************************************************************************************************
 [started TASK: copy on paravance-12.rennes.grid5000.fr]
 [started TASK: copy on paravance-13.rennes.grid5000.fr]
changed: [paravance-12.rennes.grid5000.fr]
changed: [paravance-13.rennes.grid5000.fr]

TASK [lineinfile] **************************************************************************************************************

DEBUG:root:method=delete, route=http://paravance-12.rennes.grid5000.fr:4567/pnodes, data={'type': 'remove'}
DEBUG:urllib3.connectionpool:http://paravance-12.rennes.grid5000.fr:4567 "DELETE /pnodes HTTP/1.1" 200 14922
DEBUG:enoslib.api:{'hosts': 'all', 'tasks': [{'name': 'kill -9 `ps aux|grep "distemd"|grep -v grep|sed "s/ \\{1,\\}/ /g"|cut -f 2 -d" "`|| true', 'shell': 'kill -9 `ps aux|grep "distemd"|grep -v grep|sed "s/ \\{1,\\}/ /g"|cut -f 2 -d" "`|| true'}, {'name': 'wait_for', 'wait_for': {'state': 'stopped', 'port': 4567}}, {'name': 'wait_for', 'wait_for': {'state': 'stopped', 'port': 4568}}], 'gather_facts': False, 'strategy': 'linear'}
DEBUG:enoslib.api:{'hosts': 'all', 'tasks': [{'name': 'kill -9 `ps aux|grep "distemd"|grep -v grep|sed "s/ \\{1,\\}/ /g"|cut -f 2 -d" "`|| true', 'shell': 'kill -9 `ps aux|grep "distemd"|grep -v grep|sed "s/ \\{1,\\}/ /g"|cut -f 2 -d" "`|| true'}, {'name': 'wait_for', 'wait_for': {'state': 'stopped', 'port': 4567}}, {'name': 'wait_for', 'wait_for'


PLAY [all] ************************************************************************************************************************************************

TASK [kill -9 `ps aux|grep "distemd"|grep -v grep|sed "s/ \{1,\}/ /g"|cut -f 2 -d" "`|| true] *************************************************************
 [started TASK: kill -9 `ps aux|grep "distemd"|grep -v grep|sed "s/ \{1,\}/ /g"|cut -f 2 -d" "`|| true on paravance-12.rennes.grid5000.fr]
 [started TASK: kill -9 `ps aux|grep "distemd"|grep -v grep|sed "s/ \{1,\}/ /g"|cut -f 2 -d" "`|| true on paravance-13.rennes.grid5000.fr]
changed: [paravance-12.rennes.grid5000.fr]
changed: [paravance-13.rennes.grid5000.fr]

TASK [wait_for] *******************************************************************************************************************************************
 [started TASK: wait_for on paravance-12.rennes.grid5000.fr]
 [started TASK: wait_for on paravance-13.rennes.grid5000.fr]
ok: [paravance-12.rennes.grid5000.fr]
ok: [

DEBUG:enoslib.api:{'hosts': 'paravance-12.rennes.grid5000.fr', 'tasks': [{'name': 'file', 'file': {'state': 'directory', 'dest': '/var/log/distem'}}, {'name': 'tmux new-session -d "exec distemd --verbose -d"', 'shell': 'tmux new-session -d "exec distemd --verbose -d"'}, {'name': 'wait_for', 'wait_for': {'state': 'started', 'port': 4567, 'timeout': 10}}, {'name': 'wait_for', 'wait_for': {'state': 'started', 'port': 4568, 'timeout': 10}}], 'gather_facts': False, 'strategy': 'linear'}
DEBUG:enoslib.api:{'hosts': 'paravance-12.rennes.grid5000.fr', 'tasks': [{'name': 'file', 'file': {'state': 'directory', 'dest': '/var/log/distem'}}, {'name': 'tmux new-session -d "exec distemd --verbose -d"', 'shell': 'tmux new-session -d "exec distemd --verbose -d"'}, {'name': 'wait_for', 'wait_for': {'state': 'started', 'port': 4567, 'timeout': 10}}, {'name': 'wait_for', 'wait_for': {'state': 'started', 'port': 4568, 'timeout': 10}}], 'gather_facts': False, 'strategy': 'linear'}
DEBUG:enoslib.api:Using ex


PLAY [paravance-12.rennes.grid5000.fr] ********************************************************************************************************************

TASK [file] ***********************************************************************************************************************************************
 [started TASK: file on paravance-12.rennes.grid5000.fr]
ok: [paravance-12.rennes.grid5000.fr]

TASK [tmux new-session -d "exec distemd --verbose -d"] ****************************************************************************************************
 [started TASK: tmux new-session -d "exec distemd --verbose -d" on paravance-12.rennes.grid5000.fr]
changed: [paravance-12.rennes.grid5000.fr]

TASK [wait_for] *******************************************************************************************************************************************
 [started TASK: wait_for on paravance-12.rennes.grid5000.fr]
ok: [paravance-12.rennes.grid5000.fr]

TASK [wait_for] ******************

DEBUG:root:method=post, route=http://paravance-12.rennes.grid5000.fr:4567/pnodes/, data={'target': ['paravance-12.rennes.grid5000.fr', 'paravance-13.rennes.grid5000.fr'], 'desc': {}, 'async': False}
DEBUG:urllib3.connectionpool:Resetting dropped connection: paravance-12.rennes.grid5000.fr
DEBUG:urllib3.connectionpool:http://paravance-12.rennes.grid5000.fr:4567 "POST /pnodes/ HTTP/1.1" 200 14894
DEBUG:root:method=post, route=http://paravance-12.rennes.grid5000.fr:4567/vnetworks, data={'name': 'enoslib_distem_network', 'address': '10.158.0.0/22'}
DEBUG:urllib3.connectionpool:http://paravance-12.rennes.grid5000.fr:4567 "POST /vnetworks HTTP/1.1" 200 157
DEBUG:root:method=post, route=http://paravance-12.rennes.grid5000.fr:4567/vnodes/vnode-0, data={'desc': {'host': 'paravance-12.rennes.grid5000.fr'}, 'ssh_key': {'public': 'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDmcrPaPA1Ec+vqXzFmaceeBLXIb6zwIdWd6xq1oyMMW8PDiy3Ly1yc0W8WSAUKWI9y/xEe2+5H9zGm4M41uiWksG4zbcLMCFoSh4XjLSUKrhccgF8lW7vQXnnMvJXUgbVbz

In [7]:
roles

{'server': [Host(address='10.158.0.1', alias='10.158.0.1', user='root', keyfile='/home/msimonin/workspace/repos/enoslib/docs/jupyter/keys/id_rsa', port=None, extra={'gateway': 'paravance-12.rennes.grid5000.fr', 'gateway_user': 'root'}, net_devices=set())], 'client': [Host(address='10.158.0.2', alias='10.158.0.2', user='root', keyfile='/home/msimonin/workspace/repos/enoslib/docs/jupyter/keys/id_rsa', port=None, extra={'gateway': 'paravance-12.rennes.grid5000.fr', 'gateway_user': 'root'}, net_devices=set())]}

In [8]:
networks

{'__subnet__': [<enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f3fc2534350>]}

In [9]:
roles = en.sync_info(roles, networks)

DEBUG:enoslib.api:{'hosts': 'all', 'tasks': [{'name': 'hostname', 'raw': 'hostname'}], 'gather_facts': False, 'strategy': 'linear'}
DEBUG:enoslib.api:{'hosts': 'all', 'tasks': [{'name': 'hostname', 'raw': 'hostname'}], 'gather_facts': False, 'strategy': 'linear'}
DEBUG:enoslib.api:Using extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ************************************************************************************************************************************************

TASK [hostname] *******************************************************************************************************************************************
 [started TASK: hostname on 10.158.0.1]
 [started TASK: hostname on 10.158.0.2]
changed: [10.158.0.2]
changed: [10.158.0.1]


DEBUG:enoslib.api:Using extra_vars = {'enos_action': 'check_network', 'facts_file': '/home/msimonin/workspace/repos/enoslib/docs/jupyter/_tmp_enos_/facts.json', 'ansible_python_interpreter': 'python3'}
DEBUG:enoslib.api:Running playbook /home/msimonin/workspace/repos/enoslib/enoslib/ansible/utils.yml with vars:
{'enos_action': 'check_network', 'facts_file': '/home/msimonin/workspace/repos/enoslib/docs/jupyter/_tmp_enos_/facts.json', 'ansible_python_interpreter': 'python3'}



PLAY [Gather facts for all hosts] *************************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [10.158.0.2]
ok: [10.158.0.1]

TASK [setup] **********************************************************************************************************************************************
ok: [10.158.0.2]
ok: [10.158.0.1]

PLAY [Utils functions] ************************************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [10.158.0.1]
ok: [10.158.0.2]

TASK [utils : include] *****************************************************************************************

In [11]:
results = en.run_command("nproc", roles=roles)
[(r.host,  r.stdout) for r in results]

DEBUG:enoslib.api:{'hosts': 'all', 'gather_facts': False, 'tasks': [{'name': 'nproc', 'shell': 'nproc', 'args': {}}]}
DEBUG:enoslib.api:Using extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ************************************************************************************************************************************************

TASK [nproc] **********************************************************************************************************************************************
 [started TASK: nproc on 10.158.0.1]
 [started TASK: nproc on 10.158.0.2]
changed: [10.158.0.1]
changed: [10.158.0.2]


[('10.158.0.1', '32'), ('10.158.0.2', '32')]

## Conclusion
 TODO